<a href="https://colab.research.google.com/github/yena45/28th_1_sixthSense_baco_DataAnalysis/blob/main/28th_1_sixthSense_baco_DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install --upgrade openpyxl
!pip install torch
!pip install transformers
!pip install sentencepiece==0.1.96

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292699 sha256=cf7b8feb80f465e84507cbbe8fa4c0e8585e5774e102f2b070b0b3f415c5b84b
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.10
    Uninstalling openpyxl-3.0.10:
      Successfully uninstalled openpyxl-3.0.10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-z7bglpq_/kobert-tokenizer_453daa2857eb45089baacf797577954d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-z7bglpq_/kobert-tokenizer_453daa2857eb45089baacf797577954d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=84ac02798a137698db262e69900729240be5b0baa4e7f520d5b44b672fb8d3a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-_zkp_rq3/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
#KoBERT 모델 Azure에서 모델 다운로드 서비스 중지 -> Hugging Face를 통한 모델 다운로드로 전환
#Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import gluonnlp as nlp

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
# GPU 사용
device = torch.device("cuda:0")

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
#사전 학습된 BERT 사용
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
#감정 분류를 위한 대화 음성 데이터셋
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5차년도_2차.csv", encoding='cp949')

In [ ]:
data.sample(n=2)

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
7395,5f85c2599e04b149046ccac3,해피 때문에 엄청 놀랐는데 지금은 마음이 좀 진정 되었어.,surprise,neutral,0,surprise,1,sadness,1,neutral,0,sadness,2,46,female
8878,5f90d2eb111dfd48d40ff01b,그래. 너의 말대로 음악이나 들으면서 기분 풀어야겠다. 나는 신나는 음악이 좋아.,angry,happiness,1,angry,1,sadness,1,angry,1,neutral,0,46,female


In [ ]:
#7개의 감정 class
#inplace=True는 변경 내용 원래 DataFrame에 바로 적용
data['상황'].replace(['fear', 'angry', 'sadness', 'disgust'], 0, inplace=True)
data['상황'].replace(['surprise', 'neutral'], 1, inplace=True)
data['상황'].replace('happiness', 2, inplace=True)

In [ ]:
# [발화문, 상황] data_list 생성
data_list = []
for ques, label in zip (data['발화문'], data['상황']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

In [ ]:
#데이터 분류 확인
print(data)
print(data_list[:2])

['걱정해줘서 고마워.', '0']
[['헐! 나 이벤트에 당첨 됐어.', '2'], ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '2']]


In [ ]:
#train과 test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

14530
4844


In [ ]:
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
    #return_dict=False 제거시 오류 발생 dropout(): argument 'input' (position 1) must be Tensor, not str
    #return_dict=False -> 딕셔너리가 아닌 튜플로 반환
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab

In [ ]:
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

In [ ]:
#입력 데이터셋 토큰화
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#파라미터 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tok = nlp.data.BERTSPTokenizer(tokenizer.vocab_file, vocab, lower=False)
#tokenizer -> tokenizer.vocab_file로 변경 오류 제거
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, no_deprecation_warning=True)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-22-f014d3e882b2>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/228 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.1119332313537598 train acc 0.34375
epoch 1 batch id 201 loss 0.2403576523065567 train acc 0.7553638059701493
epoch 1 train acc 0.776110197368421


<ipython-input-22-f014d3e882b2>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/76 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9543772428229665


  0%|          | 0/228 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0022938321344554424 train acc 1.0
epoch 5 batch id 201 loss 0.0340375192463398 train acc 0.9884950248756219
epoch 5 train acc 0.9889665570175439


  0%|          | 0/76 [00:00<?, ?it/s]

epoch 5 test acc 0.9541716507177034


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("0") #비추코스
            elif np.argmax(logits) == 1:
                test_eval.append("1") #무난코스
            elif np.argmax(logits) == 2:
                test_eval.append("2") #힐링코스

        return test_eval[0]


In [ ]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 42.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:
import mysql.connector

# 데이터베이스 연결 정보 설정
db_config = {
    'host': 'svc.sel4.cloudtype.app',
    'port': '32438',
    'user': 'root',
    'password': '0000',
    'database': 'baco_db'
}

# 데이터베이스 연결
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

In [ ]:
#새로운 데이터를 확인하기 위해 리뷰 테이블에 analyzed 컬럼
sql = "ALTER TABLE review ADD COLUMN analyzed TINYINT(1) DEFAULT 0;"
cursor.execute(sql)

ProgrammingError: ignored

In [ ]:
sql = "SHOW COLUMNS FROM review;"
cursor.execute(sql)

# 결과 가져오기
result = cursor.fetchall()

# 결과 출력
for row in result:
  print(row)

('review_id', 'bigint(20)', 'NO', 'PRI', None, 'auto_increment')
('content', 'varchar(255)', 'YES', '', None, '')
('date', 'date', 'YES', '', None, '')
('end_place', 'varchar(255)', 'YES', '', None, '')
('hashtag', 'varchar(255)', 'YES', '', None, '')
('start_place', 'varchar(255)', 'YES', '', None, '')
('member_id', 'bigint(20)', 'YES', 'MUL', None, '')
('route_point', 'text', 'YES', '', None, '')
('analyzed', 'int(11)', 'YES', '', None, '')


In [ ]:
sql = "SELECT content FROM review"
cursor.execute(sql)

# 결과 가져오기
result = cursor.fetchall()

# 결과 출력
for row in result:
  print(row)

('날씨가 더워서 많이 힘들어요!',)
('달리기 좋아요~',)
('그냥 그래요',)
('도로가 잘 돼있어요',)
('좋습니다',)
('11111',)
('오르막길이랑 내리막길이 너무 많아서 힘들어요. 특히 급경사가 많아 내려갈 때 위험해요. ',)
('풍경이 너무 좋아요! 자전거 도로가 있어서 안전하게 달릴 수 있습니다!',)
('너무 힘들어요. 경사가 많아서 끔찍할정도입니다. 그리고 처음엔 초록초록해서 좋지만 계속 나무 길만 나와서 지루해요',)


In [ ]:
import mysql.connector

# 데이터베이스 연결 정보 설정
db_config = {
    'host': 'svc.sel4.cloudtype.app',
    'port': '32438',
    'user': 'root',
    'password': '0000',
    'database': 'baco_db'
}

# 데이터베이스 연결
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

sql_query = "SELECT content FROM review"

def fetch_data_from_db(sql_query):
    cursor.execute(sql_query)
    result = cursor.fetchall()
    return result

data = fetch_data_from_db(sql_query)

if data:
  for row in data:
    text_to_analyze = " ".join(map(str, row))
    print("입력 문장:", text_to_analyze)
    analyze_text = predict(text_to_analyze)
    print("분석 결과:", analyze_text)

    #분석 결과 데이터베이스에 저장
    update_query = f"UPDATE review SET hashtag = '{analyze_text}' WHERE content = '{text_to_analyze}';"
    cursor.execute(update_query)
    conn.commit()
else:
  print("데이터 없음")
# 연결 닫기
cursor.close()
conn.close()

입력 문장: 날씨가 더워서 많이 힘들어요!
분석 결과: 1
입력 문장: 달리기 좋아요~
분석 결과: 2
입력 문장: 그냥 그래요
분석 결과: 0
입력 문장: 도로가 잘 돼있어요
분석 결과: 1
입력 문장: 좋습니다
분석 결과: 1
입력 문장: 11111
분석 결과: 0
입력 문장: 오르막길이랑 내리막길이 너무 많아서 힘들어요. 특히 급경사가 많아 내려갈 때 위험해요. 
분석 결과: 1
입력 문장: 풍경이 너무 좋아요! 자전거 도로가 있어서 안전하게 달릴 수 있습니다!
분석 결과: 1
입력 문장: 너무 힘들어요. 경사가 많아서 끔찍할정도입니다. 그리고 처음엔 초록초록해서 좋지만 계속 나무 길만 나와서 지루해요
분석 결과: 1
입력 문장: 경사가 많아서 끔찍합니다.
분석 결과: 1
입력 문장: 주변 풍경이 너무 이뻐요! 특히 나무가 많아서 공기도 좋고 너무 좋습니다~! 자전거 도로도 잘 구비되어있어요!
분석 결과: 1
입력 문장: 도로가 복잡해서 달리기 어려워요. 건물 풍경은 좋지만 약간 위험해요
분석 결과: 1
입력 문장: 좋아요
분석 결과: 2
입력 문장: 최악이에요
분석 결과: 0


In [ ]:
import mysql.connector
import time
# 데이터베이스 연결 정보 설정
db_config = {
    'host': 'svc.sel4.cloudtype.app',
    'port': '32438',
    'user': 'root',
    'password': '0000',
    'database': 'baco_db'
}

# 데이터베이스 연결
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()


# 실시간으로 추가되는 데이터를 가져오고 분석하여 저장
def process_realtime_data():
    while True:
        # 새로운 데이터 가져오기
        new_data_query = "SELECT content FROM review WHERE analyzed = 0"
        cursor.execute(new_data_query)
        new_data = cursor.fetchall()

        if new_data:
            for row in new_data:
                text_to_analyze = row[0]
                print("입력 문장:", text_to_analyze)

                # 분석 결과 예측
                analyze_text = predict(text_to_analyze)
                print("분석 결과:", analyze_text)

                # 분석 결과 데이터베이스에 저장
                update_query = f"UPDATE review SET hashtag = '{analyze_text}', analyzed = 1 WHERE content = '{text_to_analyze}';"
                cursor.execute(update_query)
                conn.commit()

        else:
            print("새로운 데이터 없음")
            # 적절한 대기 시간을 두고 반복 수행
        time.sleep(5)

try:
    process_realtime_data()
except KeyboardInterrupt:
    print("실시간 데이터 처리를 종료합니다.")

# 연결 닫기
cursor.close()
conn.close()

새로운 데이터 없음
새로운 데이터 없음
새로운 데이터 없음
새로운 데이터 없음
실시간 데이터 처리를 종료합니다.


In [ ]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()
sql = "SELECT hashtag FROM review"
cursor.execute(sql)

# 결과 가져오기
result = cursor.fetchall()

# 결과 출력
for row in result:
  print(row)

('1',)
('2',)
('0',)
('1',)
('1',)
('0',)
('1',)
('1',)
('1',)
('1',)
('1',)
('1',)
('2',)
('0',)
